In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import time

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup

# = = = = = = = = = = = = = = =

import re
import json

# = = = = = = = = = = = = = = =

with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

# = = = = = = = = = = = = = = =

list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/2.product_url.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['[Product] No'] = ['' if product_no == '' else int(product_no) for product_no in df_input['[Product] No'].tolist()]

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()

# = = = = = = = = = = = = = = =

def crawler(crawler_tuple):
    global api, list_dict_correct, list_series_error, count_remain

    crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

    # = = = = = = = = = = = = = = =

    try:
        crawler_series['[Product] Url'] = '' if not crawler_series['[Product] Url'] else crawler_series['[Product] Url']

        # = = = = = = = = = = = = = = =

        if crawler_series['[Product] Url'] == '':
            with semaphore_correct:
                list_dict_correct.append({'Url': crawler_series['Url']})

             # = = = = = = = = = = = = = = =

            count_retry, crawler_status = 0, True
        else:
            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_request, url_check = f'''https://api.messefrankfurt.com/service/esb_api/exhibitor-service/api/2.1/public/product/zh-CN/{crawler_series['[Product] Url'].removeprefix('https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/').removesuffix('.html')}/AUTOMECHANIKASHANGHAI/details''', crawler_series['[Product] Url']

                    resp = requests.get(url_request,
                                        headers=get_header(**{'Apikey': api}),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 401 and dict_['status'] == 401 and dict_['message'].strip() == 'API GW: Unauthorized':
                        print(f'fuck api {api}')
                        time.sleep(3)
                        with open('./api.txt', 'r') as api_txt:
                            api = api_txt.read()
                    elif resp.status_code == 200:
                        dict_ = resp.json()

                        if dict_['success'] == True:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            dict_src, list_src = {}, dict_['result']['description']['images']
            for i, src in enumerate(list_src):
                dict_src[str(i)] = f'''https://exhibitorsearch.hk.messefrankfurt.com/images/large/{src.strip()}'''

            # = = = = = = = = = = = = = = =

            if dict_['result']['description']['text']:
                soup = BeautifulSoup(dict_['result']['description']['text'], 'lxml')
                description = re.sub(r'\t+', '\n', soup.get_text()).strip()
            else:
                description = ''

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.append({'Url': crawler_series['Url'],
                                          '[Product] No': crawler_series['[Product] No'],
                                          '[Product] Name': dict_['result']['name'].strip(),
                                          '[Product] Description': description,
                                          '[Product] Picture': '',
                                          '[Product] Url': crawler_series['[Product] Url'],
                                          '[Product] Json_Src': json.dumps(dict_src)})

            # = = = = = = = = = = = = = = =

            crawler_status = True
    except:
        crawler_series['Request_Url'], crawler_series['Check_Url'] = url_request, url_check

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = = = = = = = = = = = = = =

    if count_remain % 300 == 0 and count_remain != 0:
        clear_output()
    print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['[Product] Url']}\n''')

    with semaphore_remain:
        count_remain -= 1

# = = = = = = = = = = = = = = =

pool.map(crawler, df_input.iterrows())

print('输出ing...')
print()
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Url', '[Product] No'],
                                                             ascending=[True, True],
                                                             ignore_index=True)
    df_correct.to_excel(f'''./file/3.product-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Url', '[Product] No'],
                                                           ascending=[True, True],
                                                           ignore_index=True)
    df_error.to_excel(f'''./file/product_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：1

[True] - [请求1次] - [剩余0条] - [2024/11/18 16:22:10] - https://automechanika-shanghai.hk.messefrankfurt.com/shanghai/zh-cn/exhibitor-search.detail.html/yongkang-mijia-imp--exp-coltd/mf_2_a4wc0a03divo_4061553_10000100202401.html

输出ing...

Done ~
